In [1]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
from sklearn.preprocessing import MinMaxScaler

# 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [4]:
# -----------------------------------------------------------
# 1. 데이터 로드
# -----------------------------------------------------------
# 파일 경로
file_path_usage = "C:\\Users\\82103\\Downloads\\300k_vip_승인정보.csv"
file_path_balance = "C:\\Users\\82103\\Downloads\\300k_vip_잔액정보.csv"

# 필요한 컬럼 지정 (메모리 효율화)
cols_usage = [
    '발급회원번호', '기준년월',
    '이용금액_신용_B0M', '이용건수_신용_B0M',  # 기울기 점수용
    '승인거절건수_B0M',                      # 리스크 점수용
    '이용개월수_전체_R3M', '이용개월수_전체_R6M'  # 종합이탈점수(활동성)용
]

cols_balance = [
    '발급회원번호', '기준년월',
    '잔액_B0M',                             # 기울기 점수용
    '연체잔액_B0M',                         # 리스크/연체강도 점수용
    '잔액_현금서비스_B0M', '잔액_현금서비스_B1M', # 악성부채
    '잔액_카드론_B0M', '잔액_카드론_B1M',       # 악성부채
    '연체원금_B1M', '연체원금_B2M',             # 연체강도
    '평잔_3M', '평잔_6M'                      # 자산방어
]

print("데이터 로드 중...")
# encoding='cp949'는 한글 윈도우 환경에서 흔히 사용됨
try:
    # usecols=lambda x: x in cols_... 방식을 쓰거나 리스트를 직접 전달
    # 리스트 전달 시 파일에 없는 컬럼이 있으면 에러가 나므로, lambda 방식이 안전할 수 있으나
    # 여기서는 정확한 분석을 위해 필요한 컬럼이 없으면 에러를 확인하는 것이 좋음.
    # 단, 승인거절건수 등의 위치가 확실치 않으면 일단 로드 시도.
    
    # 주의: 승인거절건수_B0M 컬럼이 실제 파일에 있는지 확인 필요.
    # 만약 에러 발생 시 컬럼명 확인 후 수정해야 함.
    df_usage = pd.read_csv(file_path_usage, usecols=lambda c: c in cols_usage)
    df_balance = pd.read_csv(file_path_balance, usecols=lambda c: c in cols_balance, encoding='utf-8')
    
    print(f"승인정보 로드 완료: {df_usage.shape}")
    print(f"잔액정보 로드 완료: {df_balance.shape}")
    
except ValueError as e:
    print("데이터 로드 중 에러 발생 (컬럼명 불일치 등):", e)
    raise

데이터 로드 중...
승인정보 로드 완료: (59364, 7)
잔액정보 로드 완료: (59364, 12)


In [5]:
# -----------------------------------------------------------
# 2. 데이터 병합
# -----------------------------------------------------------
df = pd.merge(df_usage, df_balance, on=['발급회원번호', '기준년월'], how='inner')
df.fillna(0, inplace=True)
print(f"병합 완료: {df.shape}")

병합 완료: (59364, 17)


In [6]:
# -----------------------------------------------------------
# 3. [모델 1] 기울기 기반 이탈 점수 (Slope Churn Score)
# -----------------------------------------------------------
# 3-1. 데이터 정렬
df = df.sort_values(by=['발급회원번호', '기준년월'])

# 3-2. 이동 기울기 함수
def calc_slope(y):
    if len(y) < 2 or np.sum(y) == 0:
        return 0.0
    x = np.arange(len(y))
    slope, _, _, _, _ = linregress(x, y)
    return slope

group = df.groupby('발급회원번호')

print("기울기 계산 중 (시간이 조금 걸릴 수 있습니다)...")
df['Slope_Spend'] = group['이용금액_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Balance'] = group['잔액_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)
df['Slope_Count'] = group['이용건수_신용_B0M'].rolling(window=3, min_periods=2).apply(calc_slope).reset_index(level=0, drop=True)

df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']] = df[['Slope_Spend', 'Slope_Balance', 'Slope_Count']].fillna(0)

# 3-3. 기울기 -> 점수 변환 (MinMax Scaling, 감소폭이 클수록 위험)
scaler = MinMaxScaler()
norm_scores = {}
for col in ['Slope_Spend', 'Slope_Balance', 'Slope_Count']:
    # 음수(감소)인 경우만 추출하여 양수로 변환 (가장 많이 줄어든 게 가장 큰 값 = 위험)
    neg_val = df[col].apply(lambda x: -x if x < 0 else 0)
    norm_scores[col] = scaler.fit_transform(neg_val.values.reshape(-1, 1)).flatten()

# 3-4. 가중치 적용 및 Score 1 산출
W_SPEND, W_BALANCE, W_COUNT, W_RISK = 40, 30, 20, 10

# Risk Flag: 연체나 거절이 있으면 1점
# (승인거절건수_B0M 컬럼이 존재한다고 가정)
if '승인거절건수_B0M' in df.columns:
    refusal_cnt = df['승인거절건수_B0M']
else:
    refusal_cnt = 0

df['Risk_Flag'] = np.where(
    (df['연체잔액_B0M'] > 0) | (refusal_cnt > 0),
    1, 0
)

df['Churn_Score_Slope'] = (
    (norm_scores['Slope_Spend'] * W_SPEND) +
    (norm_scores['Slope_Balance'] * W_BALANCE) +
    (norm_scores['Slope_Count'] * W_COUNT) +
    (df['Risk_Flag'] * W_RISK)
)
df['Churn_Score_Slope'] = df['Churn_Score_Slope'].round(1)

print("Slope Score 계산 완료")

기울기 계산 중 (시간이 조금 걸릴 수 있습니다)...
Slope Score 계산 완료


In [7]:
# -----------------------------------------------------------
# 4. [모델 2] 종합 이탈 점수 (Total Churn Score)
# -----------------------------------------------------------

# [부정 1] 점수_악성부채
df['점수_악성부채'] = (
    ((df['잔액_현금서비스_B0M'] - df['잔액_현금서비스_B1M']) / (df['잔액_현금서비스_B1M'] + 1) * 1.5) +
    ((df['잔액_카드론_B0M'] - df['잔액_카드론_B1M']) / (df['잔액_카드론_B1M'] + 1) * 1.0)
)

# [부정 2] 점수_연체강도
df['점수_연체강도'] = (
    (3.0 * df['연체잔액_B0M']) + 
    (2.0 * df['연체원금_B1M']) + 
    (1.0 * df['연체원금_B2M'])
)

# [긍정 1] 점수_활동성 (높을수록 좋음 -> 나중에 뺌)
df['점수_활동성'] = (
    ((df['이용개월수_전체_R3M'] * 2) - df['이용개월수_전체_R6M']) / (df['이용개월수_전체_R6M'] + 1) * 100
)

# [긍정 2] 점수_자산방어 (높을수록 좋음 -> 나중에 뺌)
df['점수_자산방어'] = (
    df['평잔_3M'] / (df['평잔_6M'] + 1) * 10
)

# 종합 이탈 점수 (Raw) 계산
df['Total_Churn_Raw'] = (
    (df['점수_악성부채'] + df['점수_연체강도']) - 
    (df['점수_활동성'] + df['점수_자산방어'])
)

# -----------------------------------------------------------
# 5. 점수 통합 (Ensemble)
# -----------------------------------------------------------
# 5-1. 모델 2 점수 스케일링 (0~100)
scaler_total = MinMaxScaler(feature_range=(0, 100))
df['Churn_Score_Total_Scaled'] = scaler_total.fit_transform(df[['Total_Churn_Raw']])

# 5-2. 최종 점수 산출 (5:5 가중치)
df['Final_Combined_Score'] = (
    (df['Churn_Score_Slope'] * 0.5) +
    (df['Churn_Score_Total_Scaled'] * 0.5)
)
df['Final_Combined_Score'] = df['Final_Combined_Score'].round(1)

print("최종 통합 점수 산출 완료")

최종 통합 점수 산출 완료


In [13]:
# -----------------------------------------------------------
# 6. 결과 확인
# -----------------------------------------------------------
target_month = df['기준년월'].max()
df_final_view = df[df['기준년월'] == target_month].copy()

cols_view = [
    '발급회원번호', '기준년월', 'Slope_Spend', 'Slope_Balance', 'Slope_Count', 'Risk_Flag',
    'Churn_Score_Slope', 'Churn_Score_Total_Scaled',
    'Final_Combined_Score'
]

print(f"\n[{target_month} 기준 최종 이탈 점수 상위 20명]")
print(df_final_view[cols_view].sort_values('Final_Combined_Score', ascending=False).head(20))

# 저장 (옵션)
df[cols_view].to_csv("final_churn_score_result.csv", index=False, encoding='utf-8-sig')


[201812 기준 최종 이탈 점수 상위 20명]
            발급회원번호    기준년월  Slope_Spend  Slope_Balance  Slope_Count  Risk_Flag  Churn_Score_Slope  Churn_Score_Total_Scaled  Final_Combined_Score
55741  SYN_1887015  201812    -589020.5      -573019.5         -0.5          0               14.4                 25.222513                  19.8
55107  SYN_1701118  201812          0.0      -122746.5        -10.5          1               19.6                 19.174197                  19.4
52734   SYN_970882  201812    -102456.5        16698.5         -0.5          1               12.5                 22.573917                  17.5
54069  SYN_1388836  201812          0.0        50113.0         -2.0          1               11.7                 18.038768                  14.9
56443  SYN_2092230  201812          0.0         -614.0         -0.5          1               10.4                 14.554393                  12.5
58116  SYN_2591811  201812     -35766.5       125937.0          0.5          0                0